# Haber Process Equation

The Haber process is used for the industrial production of ammonia from nitrogen and hydrogen gases:

$$\mathrm{N_2(g) + 3H_2(g) \rightleftharpoons 2NH_3(g)}$$

This is an equilibrium reaction where:
- Nitrogen gas (N₂) reacts with hydrogen gas (H₂) to produce ammonia (NH₃)
- The reaction is exothermic (releases heat)
- The forward reaction is favored by high pressure and low temperature
- A catalyst (typically iron) is used to increase the reaction rate

## Equilibrium Constant

The equilibrium constant (K) for this reaction is:

$$K_c = \frac{[NH_3]^2}{[N_2][H_2]^3}$$

For gas-phase reactions, we can also express this in terms of partial pressures:

$$K_p = \frac{(P_{NH_3})^2}{(P_{N_2})(P_{H_2})^3}$$

The relationship between K_p and K_c is:

$$K_p = K_c(RT)^{\Delta n}$$

Where:
- $\Delta n$ is the change in moles of gas (2 - 4 = -2 for this reaction)
- $R$ is the gas constant
- $T$ is the temperature in Kelvin

## Temperature Dependence

The temperature dependence of the equilibrium constant follows the van't Hoff equation:

$$\ln\frac{K_2}{K_1} = -\frac{\Delta H^{\circ}}{R}\left(\frac{1}{T_2} - \frac{1}{T_1}\right)$$

Where:
- $\Delta H^{\circ}$ is the standard enthalpy change of the reaction
- $K_1$ and $K_2$ are equilibrium constants at temperatures $T_1$ and $T_2$

Since the Haber process is exothermic, the equilibrium constant decreases with increasing temperature.



In [45]:
import numpy as np
from scipy.optimize import root

def calculate_equilibrium_pressures(P_N2_initial, P_H2_initial, P_NH3_initial, temperature_K):
    """
    Calculate the equilibrium partial pressures for the Haber process:
    N2(g) + 3H2(g) ⇌ 2NH3(g)
    
    Parameters:
    -----------
    P_N2_initial : float
        Initial partial pressure of N2 in atm.
    P_H2_initial : float
        Initial partial pressure of H2 in atm.
    P_NH3_initial : float
        Initial partial pressure of NH3 in atm.
    temperature_K : float
        Temperature in Kelvin.
    
    Returns:
    --------
    dict
        Dictionary containing the equilibrium partial pressures for N2, H2, and NH3,
        along with the equilibrium constant Kp and the extent of reaction.
    """
    # Reference conditions and equilibrium constant at T_ref
    T_ref = 298  # K (25°C)
    log_K_ref = np.log(6.8e5)  # Natural logarithm of equilibrium constant at reference temperature
    
    # Standard enthalpy change for the Haber process (J/mol) -- exothermic reaction
    delta_H = -92.4e3
    
    # Gas constant in J/(mol·K)
    R = 8.314
    
    # Calculate log_Kp using the van't Hoff equation:
    # log_Kp - log_K_ref = - (delta_H/R) * (1/temperature_K - 1/T_ref)
    log_Kp = log_K_ref - (delta_H / R) * (1/temperature_K - 1/T_ref)
    
    def equations(x):
        # x[0] is the extent of reaction (ξ)
        P_N2 = P_N2_initial - x[0]
        P_H2 = P_H2_initial - 3 * x[0]
        P_NH3 = P_NH3_initial + 2 * x[0]
        
        # Avoid division by zero or negative pressures by returning a large penalty value.
        if P_N2 <= 0 or P_H2 <= 0:
            return [1e20]
        
        # Equilibrium relation using logarithms for numerical stability
        # log(P_NH3^2 / (P_N2 * P_H2^3)) = log(P_NH3^2) - log(P_N2) - log(P_H2^3) = log_Kp
        log_actual = 2 * np.log(P_NH3) - np.log(P_N2) - 3 * np.log(P_H2)
        return [log_actual - log_Kp]
    
    # Initial guess for the extent of reaction
    initial_guess = [0.0]
    
    # Solve for the extent of reaction.
    solution = root(equations, initial_guess)
    if not solution.success:
        raise RuntimeError("Equilibrium solver did not converge: " + solution.message)
    
    extent = solution.x[0]
    
    # Calculate final partial pressures
    P_N2_final = P_N2_initial - extent
    P_H2_final = P_H2_initial - 3 * extent
    P_NH3_final = P_NH3_initial + 2 * extent
    
    return {
        "N2": P_N2_final,
        "H2": P_H2_final,
        "NH3": P_NH3_final,
        "log_Kp": log_Kp,
        "extent_of_reaction": extent
    }


In [47]:

result = calculate_equilibrium_pressures(1.0, 3.0, 1e-6, 700)
print("Equilibrium partial pressures (atm):")
print(f"N2: {result['N2']:.4f}")
print(f"H2: {result['H2']:.4f}")
print(f"NH3: {result['NH3']:.4f}")
print(f"Equilibrium constant Kp: {np.exp(result['log_Kp']):.4e}")


Equilibrium partial pressures (atm):
N2: 0.9562
H2: 2.8687
NH3: 0.0875
Equilibrium constant Kp: 3.3955e-04


In [53]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

# Create interactive widgets
P_total_slider = widgets.FloatSlider(
    value=4.0,
    min=0.5,
    max=20.0,
    step=0.5,
    description='Total initial pressure (atm):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

P_N2_slider = widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=10.0,
    step=0.1,
    description='P(N₂) initial:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

P_H2_slider = widgets.FloatSlider(
    value=3.0,
    min=0.3,
    max=30.0,
    step=0.3,
    description='P(H₂) initial:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

P_NH3_slider = widgets.FloatSlider(
    value=1e-6,
    min=0.0,
    max=1.0,
    step=0.01,
    description='P(NH₃) initial:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

temp_slider = widgets.IntSlider(
    value=700,
    min=400,
    max=1000,
    step=10,
    description='Temperature (K):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

output = widgets.Output()

# Function to update the plot
def update_plot(P_total, P_N2, P_H2, P_NH3, temp):
    with output:
        output.clear_output(wait=True)
        
        try:
            # Calculate scaling factor to maintain ratios but adjust to total pressure
            current_total = P_N2 + P_H2 + P_NH3
            scale_factor = P_total / current_total if current_total > 0 else 1.0
            
            # Scale the partial pressures
            scaled_P_N2 = P_N2 * scale_factor
            scaled_P_H2 = P_H2 * scale_factor
            scaled_P_NH3 = P_NH3 * scale_factor
            
            result = calculate_equilibrium_pressures(scaled_P_N2, scaled_P_H2, scaled_P_NH3, temp)
            
            # Calculate total final pressure
            total_final_pressure = result['N2'] + result['H2'] + result['NH3']
            
            # Create bar chart
            fig, ax = plt.subplots(figsize=(10, 6))
            
            # Data for plotting
            species = ['N₂', 'H₂', 'NH₃', 'Total']
            initial_pressures = [scaled_P_N2, scaled_P_H2, scaled_P_NH3, P_total]
            final_pressures = [result['N2'], result['H2'], result['NH3'], total_final_pressure]
            
            # Set positions for bars
            x = np.arange(len(species))
            width = 0.35
            
            # Create bars
            ax.bar(x - width/2, initial_pressures, width, label='Initial')
            ax.bar(x + width/2, final_pressures, width, label='Equilibrium')
            
            # Add labels and title
            ax.set_ylabel('Partial Pressure (atm)')
            ax.set_title(f'Haber Process Equilibrium at {temp} K')
            ax.set_xticks(x)
            ax.set_xticklabels(species)
            ax.legend()
            
            # Add text with equilibrium constant
            Kp = np.exp(result['log_Kp'])
            ax.text(0.5, 0.95, f'Kp = {Kp:.4e}', transform=ax.transAxes, 
                    ha='center', va='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
            
            # Add text with extent of reaction
            ax.text(0.5, 0.87, f'Extent of reaction = {result["extent_of_reaction"]:.4f}', 
                    transform=ax.transAxes, ha='center', va='top', 
                    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
            
            plt.tight_layout()
            plt.show()
            
            # Print numerical results
            print("Equilibrium partial pressures (atm):")
            print(f"N₂: {result['N2']:.4f}")
            print(f"H₂: {result['H2']:.4f}")
            print(f"NH₃: {result['NH3']:.4f}")
            print(f"Total final pressure: {total_final_pressure:.4f}")
            print(f"Equilibrium constant Kp: {Kp:.4e}")
            print(f"Extent of reaction: {result['extent_of_reaction']:.4f}")
            
        except Exception as e:
            print(f"Error: {str(e)}")
            plt.close()

# Interactive function that updates when sliders change
interactive_plot = widgets.interactive(
    update_plot,
    P_total=P_total_slider,
    P_N2=P_N2_slider,
    P_H2=P_H2_slider,
    P_NH3=P_NH3_slider,
    temp=temp_slider,
)

# Display the widgets and output
display(interactive_plot)
display(output)

# Initialize the plot
update_plot(P_total_slider.value, P_N2_slider.value, P_H2_slider.value, P_NH3_slider.value, temp_slider.value)


interactive(children=(FloatSlider(value=4.0, continuous_update=False, description='Total initial pressure (atm…

Output()